In [1]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
plt.rcParams['font.family'] = 'NanumBarunGothic'
plt.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 폴더
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)
    
import tensorflow as tf
import numpy as np

C:\Users\USER\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import tensorflow as tf
import numpy as np
import pickle

height = 32
width = 32
channels = 3
n_inputs = height * width * 3

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 8
dropout_rate = 0


reset_graph()

with tf.name_scope("inputs"):
    
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    
    conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad, name="conv1")
    bn1 = tf.layers.batch_normalization(conv1, training=True, momentum=0.9)
    bn1_act = tf.nn.selu(bn1)
    conv1_drop = tf.layers.dropout(bn1_act, dropout_rate, training=True)
    
    conv3 = tf.layers.conv2d(conv1_drop, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad, name="conv3")
    bn2 = tf.layers.batch_normalization(conv3, training=True, momentum=0.9)
    bn2_act = tf.nn.selu(bn2)
    conv3_drop = tf.layers.dropout(bn2_act, dropout_rate, training=True)



with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv3_drop, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 8 * 8])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.selu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [3]:
with open('D:\workspace/data/AffectNet/x_train.pickle','rb') as f:
    X_train = np.array(pickle.load(f))
with open('D:\workspace/data/AffectNet/x_test.pickle','rb') as f:
    X_test = np.array(pickle.load(f))
with open('D:\workspace/data/AffectNet/y_train.pickle','rb') as f:
    y_train = np.array(pickle.load(f))
with open('D:\workspace/data/AffectNet/y_test.pickle','rb') as f:
    y_test = np.array(pickle.load(f))
X_train = X_train.astype(np.float32).reshape(-1, 32*32*3) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 32*32*3) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]
print(y_train.shape)
print(X_train.shape)
print(y_train[1])

(19000,)
(19000, 3072)
1


In [4]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [5]:
n_epochs = 3
batch_size = 100
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)



with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "배치 데이터 정확도:", acc_batch, "검증 세트 정확도:", acc_val)

    acc_test = accuracy.eval(feed_dict={X: X_test,
                                        y: y_test})
    print("테스트 세트에서 최종 정확도:", acc_test)
        
    save_path = saver.save(sess, "./my_mnist_model")

0 배치 데이터 정확도: 0.19 검증 세트 정확도: 0.2192
1 배치 데이터 정확도: 0.3 검증 세트 정확도: 0.2398
2 배치 데이터 정확도: 0.23 검증 세트 정확도: 0.2356


InvalidArgumentError: First dimension of predictions 3000 must match length of targets 24000
	 [[Node: eval/in_top_k/InTopKV2 = InTopKV2[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](output/output/BiasAdd, _arg_inputs/y_0_1, eval/in_top_k/InTopKV2/k)]]

Caused by op 'eval/in_top_k/InTopKV2', defined at:
  File "C:\Users\USER\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\USER\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\USER\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\USER\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\USER\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177,